In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../data/curated/2023_prob')

In [3]:
df

,match_id,player_first_name,player_last_name,team_name,0,1,2,3
player_id,,,,,,,,
CD_I1000223,10007,Liam,Baker,Richmond,0.996487,0.002483,0.000929,0.000102
CD_I1002245,10007,Noah,Balta,Richmond,0.999040,0.000734,0.000210,0.000016
CD_I993993,10007,Shai,Bolton,Richmond,0.958959,0.024597,0.013378,0.003067
CD_I295203,10007,Nathan,Broad,Richmond,0.996505,0.002441,0.000952,0.000102
CD_I270896,10007,Trent,Cotchin,Richmond,0.993833,0.004317,0.001632,0.000219
...,...,...,...,...,...,...,...,...
CD_I294305,10199,Lachie,Whitfield,GWS Giants,0.602763,0.146600,0.152148,0.098489
CD_I1008139,10199,Toby,Bedford,GWS Giants,0.991778,0.005587,0.002312,0.000323
CD_I290675,10199,Daniel,Lloyd,GWS Giants,0.988751,0.007658,0.003161,0.000431


In [4]:
match_ids = df.match_id.unique()

In [5]:
def standardise_votes(df):
    
    ini_sum_1 = sum(df['1'])
    ini_sum_2 = sum(df['2'])
    ini_sum_3 = sum(df['3'])
    
    df['new_3'] = df['3'] / ini_sum_3
    df['new_2'] = df['2'] / ini_sum_2
    df['new_1'] = df['1'] / ini_sum_1

    df['new_0'] = 1 - (df['new_1'] + df['new_2'] + df['new_3'])
    
    df['exp_votes'] = 0*df['new_0'] + 1*df['new_1'] + 2*df['new_2'] + 3*df['new_3']
    
    
    return df[['player_id', 'match_id', 'player_first_name', 'player_last_name', 'team_name', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes']]

In [6]:
final_df = pd.DataFrame(columns=['player_id', 'match_id', 'player_first_name', 'player_last_name', 'team_name', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes'])


for idx in match_ids:
    
    df_match = df[df['match_id'] == idx].reset_index() 
    
    final_df = pd.concat([final_df, standardise_votes(df_match)])

In [7]:
final_df

,player_id,match_id,player_first_name,player_last_name,team_name,new_0,new_1,new_2,new_3,exp_votes
0,CD_I1000223,10007,Liam,Baker,Richmond,0.995547,0.002930,0.001302,0.000221,0.006197
1,CD_I1002245,10007,Noah,Balta,Richmond,0.998804,0.000867,0.000294,0.000035,0.001560
2,CD_I993993,10007,Shai,Bolton,Richmond,0.945568,0.029028,0.018752,0.006653,0.086490
3,CD_I295203,10007,Nathan,Broad,Richmond,0.995564,0.002881,0.001334,0.000221,0.006212
4,CD_I270896,10007,Trent,Cotchin,Richmond,0.992144,0.005094,0.002287,0.000474,0.011092
...,...,...,...,...,...,...,...,...,...,...
41,CD_I294305,10199,Lachie,Whitfield,GWS Giants,0.496011,0.138360,0.170193,0.195435,1.065053
42,CD_I1008139,10199,Toby,Bedford,GWS Giants,0.991501,0.005273,0.002587,0.000640,0.012366
43,CD_I290675,10199,Daniel,Lloyd,GWS Giants,0.988382,0.007227,0.003536,0.000854,0.016863
44,CD_I1008123,10199,Jake,Riccardi,GWS Giants,0.995192,0.003148,0.001401,0.000258,0.006724


In [8]:
final_df.to_parquet('../data/curated/adjusted_probabilities_2023')